In [ ]:
import math
import random
import time
from graph.graph import read_graph_from_csv

In [ ]:
from stable_matching.TaskWorker import outward_satisfactory,overall_satisfactory,gamma_workers,individual_rationality_tasks,waste_pairwise,calculate_influence_workers
def estimate(Tasks, Workers, ise=False):
    if ise:
        for task_ in Tasks:
            task_.estimate()
    result_dict = {'fairness-pairwise': outward_satisfactory(Tasks, Workers, ise),
                   # 'overall_satisfactory': overall_satisfactory(tasks,workers,ise),
                   'individual_rationality': individual_rationality_tasks(Tasks),
                   'waste-pairwise': waste_pairwise(Tasks, Workers)}
    Sum = 0
    Max = 0
    Min = 1000
    for t_ in Tasks:
        if len(t_.students()) <=0:
            break
        q = calculate_influence_workers(t_.students(), t_.G, t_.values)
        Sum += q
        if q> Max:
            Max = q
        if q< Min:
            Min = q
    result_dict['average-density'] = Sum/len(Tasks)
    result_dict['maximum-density'] = Max
    result_dict['minimum-density'] = Min
    return result_dict

In [ ]:
graph_file = '../graphs/dash/dash.csv'

In [ ]:
m = 5 # number of tasks
n = 100 # number of candidate workers

initialize workers by their ids

In [ ]:
from stable_matching.TaskWorker import Worker

g = read_graph_from_csv(graph_file,0)
workers = []
worker_ids = random.sample(list(g.nodes.keys()),n)
for i in range(n):
    workers.append(Worker(idx = worker_ids[i]))
del g

initialize tasks : (ids, budget, reverse reachable set, Q)

In [ ]:
from graph.QIM import sampling
from utils.funcs import max_k
from stable_matching.TaskWorker import Task

mu = 1.0
sigma = 0.1

# allocate budget to every task, sum of all budget is total_budget
avg_budget  = 3
max_variance = math.ceil(avg_budget/2)

budgets = [random.randint(avg_budget-max_variance,avg_budget+max_variance) for _ in range(m)]
costs = {}
values ={}
Q = [0] * m
g = read_graph_from_csv(graph_file,0)
for i in range(m):
    # generate cost of all candidate worker, task cost is generate from a trunc gaussian distribution mu = 1.0, sigma = 0.1, lb = 0.5, ub = 1.5
    X = [random.uniform(0.1,2.0) for _ in range(n)]
    costs[i] = {}
    for j in range(n):
        costs[i][workers[j]] = X[j]
        
    # values of all workers
    values[i] = {}
    
    for v in g.nodes:
        values[i][v] = random.uniform(0.0, 1.0)
        Q[i] += values[i][v]
        

In [ ]:
from tqdm import tqdm
tasks = []
graph_ids = random.sample(range(100),m)
with tqdm(total = m * 100, desc='generate tasks', leave=True, ncols=100, unit='B', unit_scale=True) as pbar:
    for i in range(m):
        start = time.time()
        G = read_graph_from_csv(graph_file,graph_ids[i])
        budget = budgets[i]
        # generate hyper graph of reverse reachable set in graph G
        k = max_k(budget, costs[i])
        RR = sampling(graph=G, C=worker_ids, k=k, delta=1/n, epsilon=0.01, values=values[i], method= 'sq')
        pbar.set_postfix({'task': i,'time used':time.time()-start})
    
        #initialize tasks
        tasks.append(Task(idx=i, budget=budget, R=RR, Q=Q[i]))
        tasks[i].initialize(costs[i])
        tasks[i].set_graph(G, values[i])
        pbar.update(100)

In [ ]:
for worker in workers:
    value_dict = {}
    for t in tasks:
        value_dict[t] = random.random()
    worker.set_preference(value_dict)

In [ ]:
from stable_matching.stableMatching import generalized_da
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
    task.set_choice_max_cover()
generalized_da(tasks,workers)
estimate(tasks,workers)

In [ ]:
from stable_matching.stableMatching import generalized_da
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
    task.set_choice_budget()
generalized_da(tasks,workers)
estimate(tasks, workers)

In [ ]:
from stable_matching.stableMatching import generalized_da
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
    task.set_choice_matroid(6)
generalized_da(tasks,workers)
estimate(tasks,workers)

In [ ]:
from stable_matching.stableMatching import heuristic
for worker in workers:
    worker.refresh()
for task in tasks:
    task.refresh()
heuristic(tasks,workers,1000)

In [ ]:
estimate(tasks,workers)